In [1]:
import pandas as pd
import numpy as np
import datetime as dt 
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import matplotlib.pyplot as plt
import networkx as nx
from mlxtend.preprocessing import TransactionEncoder
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns

In [120]:
# Read csv files
df = pd.read_csv('NOVAIMS_MAA_final.csv')

In [121]:
df.head()

,Unnamed: 0,ProductFamily_ID,ProductCategory_ID,ProductBrand_ID,ProductName_ID,ProductPackSKU_ID,Point-of-Sale_ID,Date,Quantity,Total_Sales
0,0,16,11,306,649,1970,1,2017-03-04,2.0,1540.0
1,1,16,11,306,649,1970,1,2016-05-02,4.0,3080.0
2,2,16,11,306,649,1970,1,2016-10-24,2.0,1540.0
3,3,16,11,306,649,1970,1,2017-10-13,2.0,1620.0
4,4,16,11,306,649,1970,1,2017-10-14,2.0,1620.0


In [122]:
df.drop(labels='Unnamed: 0',axis=1,inplace=True)

# testing for 1 point-of-sale


In [52]:
sample = df[df['Point-of-Sale_ID']==292]

In [53]:
sample = sample[['ProductFamily_ID','Date']]

In [54]:
sample

,ProductFamily_ID,Date
65655000,16,2017-11-28
65655001,16,2016-12-03
65655002,16,2016-12-13
65655003,16,2016-12-15
65655004,16,2016-09-02
...,...,...
66018160,4,2016-04-17
66018161,4,2016-04-18
66018162,4,2016-04-22
66018163,4,2016-04-24


In [55]:
pt_prod = pd.pivot_table(sample, index='Date', columns='ProductFamily_ID', 
                    aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0)
pt_prod.head()

ProductFamily_ID,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,21
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-01,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
2016-01-02,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
2016-01-03,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
2016-01-04,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
2016-01-05,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0


In [56]:
pt_prod= pt_prod.loc['2016-01-01':'2016-03-31']

In [57]:
pt_prod

ProductFamily_ID,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,21
Date,,,,,,,,,,,,,,,,,,,,,
2016-01-01,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0
2016-01-02,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
2016-01-03,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0
2016-01-04,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
2016-01-05,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-03-27,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
2016-03-28,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0
2016-03-29,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0


In [58]:
# Apply the APRIORI algorithm to get frequent itemsets
# Rules supported in at least 5% of the transactions (more info at http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/)
frequent_itemsets = apriori(pt_prod, min_support=0.05, max_len = 2,  use_colnames=True)
print (len(frequent_itemsets))

210


In [59]:
rules = association_rules(frequent_itemsets,metric = "support",min_threshold = 0.0)

# Funtion to compute Zhang's rule from mlxtend association_rules output
def zhangs_rule(rules):
    PAB = rules['support'].copy()
    PA = rules['antecedent support'].copy()
    PB = rules['consequent support'].copy()
    NUMERATOR = PAB - PA*PB
    DENOMINATOR = np.max((PAB*(1-PA).values,PA*(PB-PAB).values), axis = 0)
    return NUMERATOR / DENOMINATOR

# Min_support = 0.05

In [60]:
# Compute frequent itemsets using the Apriori algorithm
frequent_departments = apriori(pt_prod, 
                            min_support = 0.05, 
                            max_len = 2, 
                            use_colnames = True)

### Advanced filtering with multiple metrics

##### Complementary

In [61]:
# Recover association rules using a minium support threshold of 0.01
rules = association_rules(frequent_departments, metric = 'support', min_threshold = 0.01)

# High Confidence and high lift - complementary products
filtered_rules = rules[(rules['confidence'] > 0.6) &
                       (rules['lift'] > 1)]

# Compute Zhang's rule
filtered_rules['zhang'] = zhangs_rule(rules)

# Print remaining rule
filtered_rules.sort_values(by='zhang', ascending= False ).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhang
86,(10),(3),0.989011,0.692308,0.692308,0.700000,1.011111,0.007608,1.025641,1.000000
227,(10),(8),0.989011,0.956044,0.956044,0.966667,1.011111,0.010506,1.318681,1.000000
270,(10),(11),0.989011,0.923077,0.923077,0.933333,1.011111,0.010144,1.153846,1.000000
148,(10),(5),0.989011,0.890110,0.890110,0.900000,1.011111,0.009781,1.098901,1.000000
286,(10),(20),0.989011,0.967033,0.967033,0.977778,1.011111,0.010627,1.483516,1.000000
202,(10),(7),0.989011,0.868132,0.868132,0.877778,1.011111,0.009540,1.078921,1.000000
82,(8),(3),0.956044,0.692308,0.681319,0.712644,1.029374,0.019442,1.070769,0.649194
166,(20),(5),0.967033,0.890110,0.879121,0.909091,1.021324,0.018355,1.208791,0.633333
220,(20),(7),0.967033,0.868132,0.857143,0.886364,1.021001,0.017631,1.160440,0.623932
144,(8),(5),0.956044,0.890110,0.868132,0.908046,1.020150,0.017148,1.195055,0.449367


##### Substitutes

In [62]:
# Substitue products

filtered_sub_rules = association_rules(frequent_departments, metric="lift", min_threshold=0.0) 

filtered_sub_rules = rules[(rules['lift'] < 1)]


filtered_sub_rules['zhang'] = zhangs_rule(filtered_sub_rules)

# Print remaining rule
filtered_sub_rules.sort_values(by='zhang', ascending= True ).head(10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhang
240,(8),(17),0.956044,0.109890,0.098901,0.103448,0.941379,-0.006159,0.992815,-0.586207
101,(3),(17),0.692308,0.109890,0.054945,0.079365,0.722222,-0.021133,0.966844,-0.555556
217,(7),(17),0.868132,0.109890,0.087912,0.101266,0.921519,-0.007487,0.990404,-0.392405
100,(17),(3),0.109890,0.692308,0.054945,0.500000,0.722222,-0.021133,0.615385,-0.301724
216,(17),(7),0.109890,0.868132,0.087912,0.800000,0.921519,-0.007487,0.659341,-0.087324
179,(6),(11),0.956044,0.923077,0.879121,0.919540,0.996169,-0.003381,0.956044,-0.080460
241,(17),(8),0.109890,0.956044,0.098901,0.900000,0.941379,-0.006159,0.439560,-0.065385
178,(11),(6),0.923077,0.956044,0.879121,0.952381,0.996169,-0.003381,0.923077,-0.047619
172,(8),(6),0.956044,0.956044,0.912088,0.954023,0.997886,-0.001932,0.956044,-0.045977
173,(6),(8),0.956044,0.956044,0.912088,0.954023,0.997886,-0.001932,0.956044,-0.045977


# Testing for the rest of the points

In [49]:
sample = df[df['Point-of-Sale_ID']< 4]

In [50]:
sample

,ProductFamily_ID,ProductCategory_ID,ProductBrand_ID,ProductName_ID,ProductPackSKU_ID,Point-of-Sale_ID,Date,Quantity,Total_Sales
0,16,11,306,649,1970,1,2017-03-04,2.0,1540.0
1,16,11,306,649,1970,1,2016-05-02,4.0,3080.0
2,16,11,306,649,1970,1,2016-10-24,2.0,1540.0
3,16,11,306,649,1970,1,2017-10-13,2.0,1620.0
4,16,11,306,649,1970,1,2017-10-14,2.0,1620.0
...,...,...,...,...,...,...,...,...,...
645315,4,34,279,577,1813,3,2016-01-17,1.0,632.0
645316,4,34,279,577,1813,3,2016-01-21,1.0,632.0
645317,4,34,279,577,1813,3,2016-04-17,1.0,650.0
645318,4,34,279,577,1813,3,2016-04-21,1.0,645.0


In [47]:
sample = sample[['ProductFamily_ID','Date']]

In [ ]:
pt_prod = pd.pivot_table(sample, index='Date', columns='ProductFamily_ID', 
                    aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0)
pt_prod.head()

In [168]:

def co_occurances (point_of_sale,quarter):

    if quarter == 1:
        start = '2019-01-01' 
        end =  '2019-03-31'
        Quarter_name= '2019_Q1'
        
    if quarter == 2: 
        start = '2019-04-01' 
        end =  '2019-06-31'
        Quarter_name= '2019_Q2'
    if quarter == 3: 
        start = '2019-07-01' 
        end =  '2019-09-31'
        Quarter_name= '2019_Q3'
    if quarter == 4: 
        start = '2018-10-01' 
        end =  '2018-12-31'
        Quarter_name= '2018_Q4'
        
    sample = df[df['Point-of-Sale_ID']==point_of_sale]
    
  

    sample = sample[['ProductFamily_ID','Date']]

    pt_prod = pd.pivot_table(sample, index='Date', columns='ProductFamily_ID', 
                    aggfunc=lambda x: 1 if len(x)>0 else 0).fillna(0)
    
    pt_prod = pt_prod.loc[start:end]
    
    frequent_departments = apriori(pt_prod, 
                            min_support = 0.05, 
                            max_len = 2, 
                            use_colnames = True)
    
    rules = association_rules(frequent_departments, metric = 'support', min_threshold = 0.01)

     # High Confidence and high lift - complementary products
    filtered_rules = rules[(rules['confidence'] > 0.6) &
                       (rules['lift'] > 1)]

    # Compute Zhang's rule
    filtered_rules['zhang'] = zhangs_rule(rules)

    # Print remaining rule
    filtered_rules= filtered_rules.sort_values(by='zhang', ascending= False ).head(10)
    complementary_products = filtered_rules
    complementary_products['POS']= point_of_sale
    complementary_products['Quarter']= Quarter_name
    complementary_products['Relationship_type']= 'Complements'
    
    filtered_sub_rules2 = association_rules(frequent_departments, metric="lift", min_threshold=0.0) 

    filtered_sub_rules2 = rules[(rules['lift'] < 1)]


    filtered_sub_rules2['zhang'] = zhangs_rule(filtered_sub_rules)

    # Print remaining rule
    filtered_sub_rules2 = filtered_sub_rules2.sort_values(by='zhang', ascending= True ).head(10)
    substitute_products = filtered_sub_rules2
    substitute_products['POS']= point_of_sale
    substitute_products['Quarter']= Quarter_name
    substitute_products['Relationship_type']= 'Substitutes' 
        
    combined = pd.concat([complementary_products.drop(columns=['leverage','conviction']), substitute_products.drop(columns=['leverage','conviction'])]) 
    return combined
    

In [169]:
co_occurances(373,1)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,zhang,POS,Quarter,Relationship_type
138,(12),(6),0.977778,0.611111,0.611111,0.625000,1.022727,1.000000,373,2019_Q1,Complements
78,(9),(3),0.955556,0.622222,0.622222,0.651163,1.046512,1.000000,373,2019_Q1,Complements
85,(12),(3),0.977778,0.622222,0.622222,0.636364,1.022727,1.000000,373,2019_Q1,Complements
40,(2),(6),0.922222,0.611111,0.600000,0.650602,1.064622,0.780423,373,2019_Q1,Complements
69,(21),(2),0.955556,0.922222,0.911111,0.953488,1.033903,0.737805,373,2019_Q1,Complements
94,(18),(3),0.833333,0.622222,0.588889,0.706667,1.135714,0.716981,373,2019_Q1,Complements
214,(9),(18),0.955556,0.833333,0.822222,0.860465,1.032558,0.709459,373,2019_Q1,Complements
199,(21),(8),0.955556,0.777778,0.766667,0.802326,1.031561,0.688406,373,2019_Q1,Complements
62,(2),(18),0.922222,0.833333,0.811111,0.879518,1.055422,0.675147,373,2019_Q1,Complements
76,(8),(3),0.777778,0.622222,0.566667,0.728571,1.170918,0.656863,373,2019_Q1,Complements


In [170]:
df_rules = pd.DataFrame([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                          index=['Antecedent', 'Consequent', 'Antecedent_Support',
                                 'Consequent_Support', 'Support', 'Confidence',
                                'Lift', 'Zhang', 'POS', 'Quarter', 'Relationship']).T
data = pd.DataFrame([])
list_point_of_sale = [91,310,375,178,307,167,373,237,221,243,292,92,280,78,272,103,282,383,359,360]
for point_of_sale in list_point_of_sale:
    for quarter in [1,2,3,4]:
        result = co_occurances(point_of_sale, quarter)
        data = data.append(result)

In [171]:
data.tail(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,zhang,POS,Quarter,Relationship_type
176,(20),(5),0.593407,0.879121,0.516484,0.870370,0.990046,-0.044444,360,2019_Q2,Substitutes
177,(5),(20),0.879121,0.593407,0.516484,0.587500,0.990046,-0.011494,360,2019_Q2,Substitutes
148,(5),(6),0.879121,0.956044,0.835165,0.950000,0.993678,NaN,360,2019_Q2,Substitutes
149,(6),(5),0.956044,0.879121,0.835165,0.873563,0.993678,NaN,360,2019_Q2,Substitutes
186,(10),(6),0.956044,0.956044,0.912088,0.954023,0.997886,NaN,360,2019_Q2,Substitutes
187,(6),(10),0.956044,0.956044,0.912088,0.954023,0.997886,NaN,360,2019_Q2,Substitutes
188,(11),(6),0.945055,0.956044,0.901099,0.953488,0.997327,NaN,360,2019_Q2,Substitutes
189,(6),(11),0.956044,0.945055,0.901099,0.942529,0.997327,NaN,360,2019_Q2,Substitutes
198,(16),(6),0.967033,0.956044,0.923077,0.954545,0.998433,NaN,360,2019_Q2,Substitutes
199,(6),(16),0.956044,0.967033,0.923077,0.965517,0.998433,NaN,360,2019_Q2,Substitutes


In [173]:
data.to_csv('Association_Rules_.csv')

In [ ]:
association rules